# 🔄 MLOps Pipelines

**Author**: Data Science Master System  
**Difficulty**: ⭐⭐⭐⭐ Advanced  
**Time**: 75 minutes  
**Prerequisites**: 26_model_deployment_api

## Learning Objectives
- Build ML pipelines with Airflow
- MLflow for experiment tracking
- Feature stores (Feast)
- Model versioning

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import json
from datetime import datetime

## 1. MLflow Experiment Tracking

In [ ]:
try:
    import mlflow
    import mlflow.sklearn
    
    mlflow.set_experiment("iris_classification")
    
    X, y = load_iris(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    with mlflow.start_run():
        # Log parameters
        mlflow.log_param("n_estimators", 100)
        mlflow.log_param("max_depth", 10)
        
        # Train model
        model = RandomForestClassifier(n_estimators=100, max_depth=10)
        model.fit(X_train, y_train)
        
        # Log metrics
        accuracy = model.score(X_test, y_test)
        mlflow.log_metric("accuracy", accuracy)
        
        # Log model
        mlflow.sklearn.log_model(model, "model")
        
    print(f"✅ Experiment logged! Accuracy: {accuracy:.4f}")
except ImportError:
    print("Install: pip install mlflow")

## 2. Airflow DAG Structure

In [ ]:
dag_code = '''
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

default_args = {
    'owner': 'data_team',
    'start_date': datetime(2024, 1, 1),
    'retries': 1
}

dag = DAG('ml_training_pipeline', default_args=default_args, schedule_interval='@daily')

def extract_data(): pass
def transform_data(): pass  
def train_model(): pass
def evaluate_model(): pass
def deploy_model(): pass

extract = PythonOperator(task_id='extract', python_callable=extract_data, dag=dag)
transform = PythonOperator(task_id='transform', python_callable=transform_data, dag=dag)
train = PythonOperator(task_id='train', python_callable=train_model, dag=dag)
evaluate = PythonOperator(task_id='evaluate', python_callable=evaluate_model, dag=dag)
deploy = PythonOperator(task_id='deploy', python_callable=deploy_model, dag=dag)

extract >> transform >> train >> evaluate >> deploy
'''

print("📋 Airflow DAG:")
print(dag_code)

## 3. Model Registry

In [ ]:
class SimpleModelRegistry:
    def __init__(self):
        self.models = {}
    
    def register(self, name, model, version, metrics):
        if name not in self.models:
            self.models[name] = []
        self.models[name].append({
            'version': version,
            'model': model,
            'metrics': metrics,
            'timestamp': datetime.now().isoformat()
        })
        print(f"✅ Registered {name} v{version}")
    
    def get_latest(self, name):
        return self.models.get(name, [])[-1] if name in self.models else None
    
    def list_versions(self, name):
        return [(m['version'], m['metrics']) for m in self.models.get(name, [])]

# Demo
registry = SimpleModelRegistry()
registry.register('iris_classifier', model, '1.0.0', {'accuracy': accuracy})
registry.register('iris_classifier', model, '1.0.1', {'accuracy': accuracy + 0.01})
print(f"\nVersions: {registry.list_versions('iris_classifier')}")

## 4. CI/CD Pipeline

In [ ]:
github_actions = '''
name: ML Pipeline
on: [push]
jobs:
  train:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v2
    - name: Set up Python
      uses: actions/setup-python@v2
    - name: Install dependencies
      run: pip install -r requirements.txt
    - name: Train model
      run: python train.py
    - name: Run tests
      run: pytest tests/
    - name: Deploy
      run: python deploy.py
'''
print("🔄 GitHub Actions Workflow:")
print(github_actions)

## 🎯 Key Takeaways
- MLflow for experiment tracking
- Airflow for orchestration
- Model registry for versioning
- CI/CD for automation

**Next**: 28_cloud_deployment.ipynb